## Introduction
If you would like to add a reaction to the databases which does not currently appear in the database you should follow these steps here. If you already have initial and final frames, you can avoid this step and instead please see https://github.com/FAIR-Chem/fairchem/blob/4164cc08695d8283714b07bbcb03ff2916a797c9/src/fairchem/applications/cattsunami/tutorial/fairchem_models_for_nebs.ipynb

## 1. Add adsorbates to the adsorbate dbs if necessary

If the adsorbates you are interested in are also absent from the adsorbate databases you will have to add them as well. If the adsorbate is very dissimilar from those present in the OC20 database, be aware that you may have higher errors due to the adsorbate being out of domain. The validation dataset has shown that there is generalization to unseen adsorbates that are similar to those in domain.

### Are the adsorbates involved in your reaction already in the adsorbate db?

In [42]:
import pickle
from fairchem.data.oc.databases.pkls import ADSORBATE_PKL_PATH

with open(ADSORBATE_PKL_PATH, "rb") as f:
    adsorbates = pickle.load(f)

In [41]:
for idx in range(len(adsorbates)):
    print(idx, adsorbates[idx][1])

0 *O
1 *H
2 *OH
3 *OH2
4 *C
5 *CO
6 *CH
7 *CHO
8 *COH
9 *CH2
10 *CH2*O
11 *CHOH
12 *CH3
13 *OCH3
14 *CH2OH
15 *CH4
16 *OHCH3
17 *C*C
18 *CCO
19 *CCH
20 *CHCO
21 *CCHO
22 *COCHO
23 *CCHOH
24 *CCH2
25 *CH*CH
26 CH2*CO
27 *CHCHO
28 CH*COH
29 *COCH2O
30 *CHO*CHO
31 *COHCHO
32 *COHCOH
33 *CCH3
34 *CHCH2
35 *COCH3
36 *OCHCH2
37 *COHCH2
38 *CHCHOH
39 *CCH2OH
40 *CHOCHOH
41 *COCH2OH
42 *COHCHOH
43 *CH2*CH2
44 *OCHCH3
45 *COHCH3
46 *CHOHCH2
47 *CHCH2OH
48 *OCH2CHOH
49 *CHOCH2OH
50 *COHCH2OH
51 *CHOHCHOH
52 *CH2CH3
53 *OCH2CH3
54 *CHOHCH3
55 *CH2CH2OH
56 *CHOHCH2OH
57 *OHCH2CH3
58 *NH2N(CH3)2
59 *ONN(CH3)2
60 *OHNNCH3
61 *NNCH3
62 *ONH
63 *NHNH
64 *NH2NH2
65 *N*NH
66 *ONNO2
67 *NO2NO2
68 *N*NO
69 *N2
70 *ONNH2
71 *NH2
72 *NH3
73 *NONH
74 *NH
75 *NO2
76 *NO
77 *N
78 *NO3
79 *OHNH2
80 *ONOH
81 *CN
82 CO*COH
83 *OCHO
84 *COOH
85 *OOH


### If not, you will have to add them.

Each adsorbate entry is a tuple containing the following information:
1. The adsorbate atoms object
2. A string representing the adsorbate and its adsorbed atom(s)
    - For monoatomic hydrogen we use `*H` where the asterisk appears just beform the binding atom(s)
    - If an adsorbate is definitively multidentate, it appears with multiple asterisks (i.e. `*CH2*CH2`)
    - This isnt super important so long as it is meaningful to you, because it is a method of querying the adsorbate from the db using the `Adsorbate` class
3. A numpy array containing the binding index/indices
    - This is the index of the atom or atoms which are expected to bind
    - For `*H` and other monoatomic adsorbates it will always be 0, for others you will have to inspect the adsorbate and pick the correct index/indices
4. A html string of the adsorption reaction with the OC20 referencing scheme to CO, H2, H2O, and N2 in the gas phase
    - This is so the reaction may be rendered on the demo website and has no other use, so do not worry about it too much
    

## 2. Set up the basic reaction db elements
All three reaction types have the same basic elements. The first few are very straight forward, and will be discussed here. Each db entry is a dictionary and the db is just a list of these dictionaries. Each dictionary contains.
1. `reaction` a string representation of the reaction. This may be used to query the db using the `Reaction` class and is used to make the entries human readable. The exact content of it does not impact anything.
2. `reaction_type` which is either desorption, dissociation, or transfer. This corresponds to what database the reaction should appear in
3. The reactant and product indices. This is different for each of the reaction classes because they have differing numbers of reactant(s) and products(s)
    - Desorptions: `reactant` and `product` are entries to the db dictionary
    - Dissociation: `reactant`, `product1`, and `product2` are entries to the db dictionary
    - Transfer: `reactant1`, `reactant2`, `product1`, and `product2` are entries to the db dictionary
    
Let's look at examples of these.

In [43]:
# you may have to change the paths
DESORPTION_PATH = "../databases/desorptions_9Aug23.pkl"
DISSOCIATION_PATH = "../databases/dissociation_reactions_22May24.pkl"
TRANSFER_PATH = "../databases/transfers_5Sept23.pkl"

with open(DESORPTION_PATH, "rb") as f:
    desorptions = pickle.load(f)
    
with open(DISSOCIATION_PATH, "rb") as f:
    dissociations = pickle.load(f)
    
with open(TRANSFER_PATH, "rb") as f:
    transfers = pickle.load(f)

In [44]:
desorptions[0]

{'reaction': '*CO -> CO(g)',
 'reaction_type': 'desorption',
 'reactant': 5,
 'product': 5,
 'idx_mapping': [{0: 0, 1: 1}],
 'edge_indices': [(0, 1)],
 'broken_edge': []}

For desorptions, the reactant and product indices are redundant. As can be seen above, `*CO` has an index of 5 in the adsorbate db, and that is its index in the desorption db.

In [45]:
dissociations[0]

{'reaction': '*OH -> *O + *H',
 'reaction_type': 'dissociation',
 'reactant': 2,
 'product1': 0,
 'product2': 1,
 'idx_mapping': [{0: 0, 1: 1}],
 'edge_indices_initial': [(0, 1)],
 'edge_indices_final': []}

Product 1 is defined as sharing the same binding atom as the reactant, so in this case it must be `*O`. As can be seen above the indices listed for the reactants and products match those in the adsorbate db.

In [46]:
transfers[0]

{'reaction': '*OH + *CH2 -> *O + *CH3',
 'reaction_type': 'transfer',
 'reactant1': 2,
 'reactant2': 9,
 'product1': 0,
 'product2': 12,
 'idx_mapping': [{0: 0, 1: 2, 2: 1, 3: 3, 4: 4},
  {0: 0, 1: 3, 2: 1, 3: 2, 4: 4},
  {0: 0, 1: 4, 2: 1, 3: 3, 4: 2}],
 'edge_indices_initial': [(0, 1), (2, 3), (2, 4)],
 'edge_indices_final': [(1, 2), (1, 3), (1, 4)]}

Product 1 must share the same binding atom as reactant 1 and product 2 must share the same binding atom as reactant 2. As can be seen above the indices listed for the reactants and products match those in the adsorbate db.

### Complete this now for the reaction you are interested in

In [ ]:
new_entry = {"reaction": ,
             "reaction_type": ,
            }

## 3. Add edges which will be enforced

As a part of the workflow to enumerate possible reactant and product configurations, we enforce that the bonds we expect are maintained and no erroneous additional bonds exist. The information in `edge_indices_initial` and `edge_indices_final` or `edge_indices` facillitate this. The functions below should automatically handle this.

In [48]:
from itertools import combinations
from ase.data import atomic_numbers, covalent_radii
import ase

def get_edges(atoms: ase.Atoms):
    """
    Get the edges for all atoms in an atoms object.

    Args:
        edge_list (list[tuples]): The edges
    """
    edge_list = []
    elements = atoms.get_chemical_symbols()
    all_combos = list(combinations(range(len(atoms)), 2))
    for combo in all_combos:
        total_distance = atoms.get_distance(combo[0], combo[1], mic=True)
        r1 = covalent_radii[atomic_numbers[elements[combo[0]]]]
        r2 = covalent_radii[atomic_numbers[elements[combo[1]]]]
        distance_ratio = total_distance / (r1 + r2)
        if distance_ratio <= 1.25:
            edge_list.append(tuple(combo))
    return edge_list

def add_edges(entry, adsorbate_db):
    if entry["reaction_type"] == "desorption":
        adsorbate = adsorbate_db[entry["reactant"]][0]
        entry["edge_indices"] = get_edges(adsorbate)
        entry["broken_edge"] = []
        
    elif entry["reaction_type"] == "dissociation":
        reactant = adsorbate_db[entry["reactant"]][0]
        entry["edge_indices_initial"] = get_edges(reactant)
        
        pdt1 = adsorbate_db[entry["product1"]][0].copy()
        pdt2 = adsorbate_db[entry["product2"]][0].copy()
        pdt2.translate([99,99,99]) # just so they dont overlap when concatenated
        
        pdts = pdt1 + pdt2
        entry["edge_indices_final"] = get_edges(pdts)
        
    elif entry["reaction_type"] == "transfer":
        rxt1 = adsorbate_db[entry["reactant1"]][0].copy()
        rxt2 = adsorbate_db[entry["reactant2"]][0].copy()
        rxt2.translate([99,99,99])
        rxts = rxt1 + rxt2
        entry["edge_indices_initial"] = get_edges(rxts)
        
        pdt1 = adsorbate_db[entry["product1"]][0].copy()
        pdt2 = adsorbate_db[entry["product2"]][0].copy()
        pdt2.translate([99,99,99])
        
        pdts = pdt1 + pdt2
        entry["edge_indices_final"] = get_edges(pdts)
        
    return entry

In [ ]:
new_entry = add_edges(new_entry, adsorbates)

## 4. Add the mapping scheme
Lastly, we have to add a mapping so the ordering of the adsorbate atoms in the initial frame match the ordering in the final frame. In the case of desorptions, this is a trivial 1:1 mapping. For transfers and dissociations it can be more complicated, particularly when there are multiple possible leaving groups. As an example, for `*CH2 -> *CH + *H` there are 2 possible hydrogens which can leave. We want to pick the one that minimizes the total distance traversed by all atoms to get from the reactant state to the product state. This happend automatically in the AutoFrame classes, but to do so we have to allow the multiple possible mappings in the database file. It is possible that this may be done algorithmically, but that was not done for this work. If you devise a way, let us know. Let's walk through an example with 2 possible mappings.

In [50]:
ex = dissociations[10]
ex

{'reaction': '*CH2*O -> *CHO + *H',
 'reaction_type': 'dissociation',
 'reactant': 10,
 'product1': 7,
 'product2': 1,
 'idx_mapping': [{0: 0, 1: 3, 2: 1, 3: 2}, {0: 0, 1: 1, 2: 3, 3: 2}],
 'edge_indices_initial': [(0, 1), (0, 2), (0, 3)],
 'edge_indices_final': [(0, 1), (0, 2)]}

In [52]:
from x3dase.visualize import view_x3d_n

view_x3d_n(adsorbates[ex["reactant"]][0])

In [54]:
pdt1 = adsorbates[ex["product1"]][0].copy()
pdt2 = adsorbates[ex["product2"]][0].copy()
pdt2.translate([2,2,2])

view_x3d_n(pdt1 + pdt2)

In [ ]:
# The reactant is:
#          H(1)
#         /
# O(3) = C(0)
#         \
#          H(2)
         
# The mapping tells us the potential indices in the product state:

#             H(3)                       H(1)
#            /                          /
#    O(2) = C(0)       or       O(2) = C(0)
#            \                         \
#            H(1)                      H(3)

# This gives a mapping of:
#             H(1:3)                           H(1:1)
#            /                                /
#    O(3:2) = C(0:0)       or       O(3:2) = C(0:0)
#            \                               \
#            H(2:1)                          H(2:3)


Therefore the mapping should be 
```
[{0: 0, 1: 3, 2: 1, 3: 2}, {0: 0, 1: 1, 2: 3, 3: 2}]
```

In [ ]:
# Add your mapping and then add it to the appropriate file!

new_entry["idx_mapping"] = []
